## Testing finalized func

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from htools import magics

In [3]:
def process1(text):
    for i in range(1_000):
        continue
    return text

In [4]:
def process2(text):
    for i in range(100):
        continue
    return text

In [5]:
def process3(text):
    for i in range(100*len(text)):
        continue
    return text

In [6]:
text = 'abc'

In [7]:
%%race
process1(text)
process2(text)
process3(text)

23.6 µs ± 2.6 µs per loop (mean ± std. dev. of 3 runs, 5 loops each)
1.58 µs ± 129 ns per loop (mean ± std. dev. of 3 runs, 5 loops each)
7.46 µs ± 779 ns per loop (mean ± std. dev. of 3 runs, 5 loops each)


## Development

In [1]:
from IPython.core.magic import cell_magic, magics_class, Magics
from IPython.core.magic_arguments import (argument, magic_arguments, 
                                          parse_argstring)

In [30]:
from htools import eprint

In [243]:
@magics_class
class FunctionRacerMagic(Magics):
    
    @cell_magic
    @magic_arguments()
    @argument('-n')
    @argument('-r')
    def race(self, line, cell):
        """"""
        args = parse_argstring(self.race, line)
        n = args.n or 5
        r = args.r or 3
        
        # Split cell into lines of code to execute.
        rows = [row for row in cell.strip().split('\n') 
                if not row.startswith('#')]
        prefix = f'%timeit -n {n} -r {r} '
        for row in rows: 
            get_ipython().run_cell(prefix + row)

In [244]:
ip = get_ipython()
ip.register_magics(FunctionRacerMagic)

In [245]:
%%race
'a'
2 + 6
'a \nb'

64.7 ns ± 34.5 ns per loop (mean ± std. dev. of 3 runs, 5 loops each)
81.8 ns ± 56.9 ns per loop (mean ± std. dev. of 3 runs, 5 loops each)
54.8 ns ± 18.5 ns per loop (mean ± std. dev. of 3 runs, 5 loops each)


In [246]:
%%race -n 4
'a'
2 + 6
'a \nb'

86.5 ns ± 56.2 ns per loop (mean ± std. dev. of 3 runs, 4 loops each)
75.5 ns ± 37.1 ns per loop (mean ± std. dev. of 3 runs, 4 loops each)
86.2 ns ± 44.3 ns per loop (mean ± std. dev. of 3 runs, 4 loops each)


In [247]:
%%race -r 6
'a'
2 + 6
'a \nb'

63.2 ns ± 31 ns per loop (mean ± std. dev. of 6 runs, 5 loops each)
62.6 ns ± 28.1 ns per loop (mean ± std. dev. of 6 runs, 5 loops each)
59.6 ns ± 35.5 ns per loop (mean ± std. dev. of 6 runs, 5 loops each)


In [248]:
%%race -n 7 -r 2
'a'
2 + 6
'a \nb'

67.1 ns ± 27.7 ns per loop (mean ± std. dev. of 2 runs, 7 loops each)
54.5 ns ± 12.5 ns per loop (mean ± std. dev. of 2 runs, 7 loops each)
86.4 ns ± 8.86 ns per loop (mean ± std. dev. of 2 runs, 7 loops each)


In [249]:
%timeit -n 3 -r 2 print('a')

a
a
a
a
a
a
35.4 µs ± 19.8 µs per loop (mean ± std. dev. of 2 runs, 3 loops each)


In [250]:
%%timeit -n 3 -r 2
print('a')

a
a
a
a
a
a
The slowest run took 4.00 times longer than the fastest. This could mean that an intermediate result is being cached.
38.8 µs ± 23.3 µs per loop (mean ± std. dev. of 2 runs, 3 loops each)


In [251]:
%%timeit -n 3 -r 2
[i for i in range(100)]

3.88 µs ± 483 ns per loop (mean ± std. dev. of 2 runs, 3 loops each)


In [252]:
%timeit -n 2 -r 1 [i for i in range(100)]

6.25 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 2 loops each)


## Test on slower functions

In [256]:
%%race
process1(text)
process2(text)

22.1 µs ± 213 ns per loop (mean ± std. dev. of 3 runs, 5 loops each)
1.72 µs ± 168 ns per loop (mean ± std. dev. of 3 runs, 5 loops each)


In [257]:
%%race -n 6 -r 4
process1(text)
process2(text)

20.8 µs ± 137 ns per loop (mean ± std. dev. of 4 runs, 6 loops each)
2.4 µs ± 59.5 ns per loop (mean ± std. dev. of 4 runs, 6 loops each)


In [259]:
%%race
process1(text)
process2(text)
process3(text)

21.7 µs ± 368 ns per loop (mean ± std. dev. of 3 runs, 5 loops each)
2.82 µs ± 133 ns per loop (mean ± std. dev. of 3 runs, 5 loops each)
7.93 µs ± 84.7 ns per loop (mean ± std. dev. of 3 runs, 5 loops each)


In [261]:
%%race

# Testing handling comments and whitespace: func1
process1(text)

# func2
process2(text)

# func3
process3(text)

20.5 µs ± 109 ns per loop (mean ± std. dev. of 3 runs, 5 loops each)
1.58 µs ± 149 ns per loop (mean ± std. dev. of 3 runs, 5 loops each)
4.55 µs ± 159 ns per loop (mean ± std. dev. of 3 runs, 5 loops each)


In [262]:
def many_args_func(a, b, c, d, e, f, g, h, i, j, k, l, m):
    for i in range(500):
        continue
    return a

In [266]:
many_args_func(a=1, b=2, c=3, d=4, e=5, f=6, g=7, h=8, i=9, 
               j=10, k=11, l=12, m=13)

1

In [267]:
%%race
many_args_func(a=1, b=2, c=3, d=4, e=5, f=6, g=7, h=8, i=9, 
               j=10, k=11, l=12, m=13)

SyntaxError: unexpected EOF while parsing (<unknown>, line 1)

SyntaxError: invalid syntax (<unknown>, line 1)

In [263]:
%%race
many_args_func(*range(13))

9.52 µs ± 335 ns per loop (mean ± std. dev. of 3 runs, 5 loops each)
